In [1]:
import sys
sys.path.append("../src")
from data import make_dataset

In [2]:
from working_path import PROJECT_PATH

## Dowload do conjunto de dados

O Download dos dados é feito caso ele não exista na pasta de dados brutos.

In [3]:
columns = ["age", "workclass", "fnlwgt", "education", "education_num",
           "marital_status", "occupation", "relationship", "race", "sex",
           "capital_gain", "capital_loss", "hours_per_week", "native_country",
           "salary"]
dataset_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/adult/{}"

    

In [4]:
make_dataset.download_dataset(request_url=dataset_url, download_filename="adult.data",
                     output_filename="train_data.csv", columns_name=columns)
make_dataset.download_dataset(request_url=dataset_url, download_filename="adult.test",
                     output_filename="test_data.csv", columns_name=columns)

## Tratamento dos dados

A Pipeline de tratamento dos dados é bem simplicada, duas novas características são constrídas a partir das existentes, algumas são removidas e as categóricas restantes são transformada em numéricas.

In [5]:
import pandas as pd

In [6]:
from features import build_features, features_encoder

In [7]:
datasets = {}
for dataset_id in ["train", "test"]:
    dataset = pd.read_csv(f"{PROJECT_PATH}/data/raw/{dataset_id}_data.csv")
    dataset_ = dataset.copy()
    dataset = build_features.add_new_features(dataset.copy())
    dataset = build_features.drop_features(dataset.copy())
    dataset = build_features.drop_nan_values(dataset.copy())
    dataset = build_features.make_target(dataset.copy())
    dataset = build_features.drop_features(dataset.copy(), columns=["salary"])
    datasets[dataset_id] = dataset

/home/nicolehervis/others/Projects/classification-project/notebooks/../src/features/build_features.py:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataset["salary"] = dataset.salary.str.replace(".", "")


A parte da transformação dos dados categóricos em classe numérica foi feita ocm base nos dados de treino e aplicada aos dados de teste, nessa etapa optou-se por manter os dados faltantes como uma nova categoria.

In [9]:
cod = features_encoder.fit_encoder(datasets["train"])

OrdinalEncoder()


In [10]:
cod.categories_

[array([' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private',
        ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'],
       dtype=object),
 array([' Divorced', ' Married-AF-spouse', ' Married-civ-spouse',
        ' Married-spouse-absent', ' Never-married', ' Separated',
        ' Widowed'], dtype=object),
 array([' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair',
        ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners',
        ' Machine-op-inspct', ' Other-service', ' Priv-house-serv',
        ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support',
        ' Transport-moving'], dtype=object),
 array([' Husband', ' Not-in-family', ' Other-relative', ' Own-child',
        ' Unmarried', ' Wife'], dtype=object),
 array([' Female', ' Male'], dtype=object),
 array([' Amer-Indian-Eskimo', ' Asian-Pac-Islander', ' Black', ' Other',
        ' White'], dtype=object)]

In [11]:
for dataset_id in ["train", "test"]:
    dataset = features_encoder.transform_features_with_encoder(datasets[dataset_id])
    dataset.to_csv(f"{PROJECT_PATH}/data/processed/{dataset_id}_data.csv", index=False)

Os dados tratados são salvos na pasta de dados processados

## Balanceamento do conjunto de treinamento

Devido ao desbalanceamento significativo no conjunto de dados optou-se por fazer em um primeiro momento um _under-sampling_ da classe majoritória para 75% da quantidade de dados total e após isso um _over-sampling_ da classe minoritária até se igualar a quatidade de dados da majoritária.

In [12]:
from data import resampling

In [13]:
train_data = pd.read_csv(f"{PROJECT_PATH}/data/processed/train_data.csv")

In [14]:
train_data.shape

(32561, 13)

In [15]:
balance = resampling.Resampling()

In [16]:
X_balanced, y_balanced = balance.apply_pipeline(X=train_data.drop(columns="target"),
                                                y = train_data["target"])

In [17]:
data_balanced = X_balanced.copy()
data_balanced["target"] = y_balanced

In [18]:
data_balanced.shape

(37080, 13)

In [19]:
data_balanced.to_csv(f"{PROJECT_PATH}/data/processed/train_data_balanced.csv", index=False)

Com essa abordagem de cerca de 32 mil exemplos obtém-se um pouco mais de 37 mil. A escolha por mesclar as duas metodologias de reamostragem é para que não predomine muitos exemplos sintéticos no caso de escolher apenas pela técnica de aumento de dados e nem perder muitos dados optando apenas pela redução dos dados, isso também permite um aumento mais controlado do conjunto de dados, minim,izando o custo de processamento.

## Modelo

Para modelagem,o algoritmo escolhido foi o Random Forest devido a sua capacidade em se ajustar a diversos problemas e como não é um método tão sensível a escala dos dados e consegue dar peso de forma adaptativa as características mais importnates, já que não foi possível fazer um processo mais profundo nessa curadoria dos dados.

In [20]:
method = "random_forest"

In [21]:
from models.model import Modeling

In [22]:
data = pd.read_csv(f"{PROJECT_PATH}/data/processed/train_data_balanced.csv")

Primeiro foi feita uma busca de parametros em um conjunto bem delimitado para limitar o processamento. Nessa etapa utilizou-se de cros-validation com 10-fold.

In [23]:
ml = Modeling(data, method)
prts = ml.searching_parameters(method)
print(prts)

{'max_depth': 15, 'n_estimators': 50, 'random_state': 42}


Com os parametros selecionados o modelo foi treinado com toda a base de treinamento.

In [24]:
model = ml.training(parameters=prts)

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.94      0.92     18540
           1       0.94      0.90      0.92     18540

    accuracy                           0.92     37080
   macro avg       0.92      0.92      0.92     37080
weighted avg       0.92      0.92      0.92     37080



In [25]:
test_data = pd.read_csv(f"{PROJECT_PATH}/data/processed/test_data.csv")

In [26]:
Modeling.test(model, x=test_data.drop(columns="target"),
              y = test_data["target"])

Classification Report:
              precision    recall  f1-score   support

           0       0.63      0.77      0.69      3846
           1       0.92      0.86      0.89     12435

    accuracy                           0.84     16281
   macro avg       0.77      0.81      0.79     16281
weighted avg       0.85      0.84      0.84     16281



É possível perceber um melhor desempenho na classe majoriátia, mas considerando o desbalanceamento existente nos dados brutos, o balanceamento efetuado conseguiu auxiliar nessa questão.

Para melhorar o modelo além de testar outros algoritmos e abordagens é essencial rever os dados e olhar com mais cuidado as características, pensar na remoção de dados faltantes ou outro tratamento, um encoder com uma ordem que agregue um valor ordinal e a construção de características.